In [ ]:
# %load_ext rpy2.ipython
# !rm -rf RiesgosFinancieros
# !git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

In [3]:
%load_ext rpy2.ipython
!rm -rf Riesgos-Financieros
!git clone "https://github.com/act-prma/RiesgosFinancieros"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 584, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 584 (delta 150), reused 146 (delta 146), pack-reused 411
Receiving objects: 100% (584/584), 25.89 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (372/372), done.


In [259]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.18.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 154512 bytes (150 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [260]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 


# Definición de parámetros para valorar:
Acciones
Bondes D
SWAPS
Futuros
Opciones

Cargamos parámetros para valoración:

In [261]:
%%R

fval=as.Date("20220331",format="%Y%m%d")#Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
#setwd(direc)	

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_eq=c(-5000,1000,1200) #monto inicial invertido en acciones

#Cargar los símbolos de yahoo finance para FX
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_fx=c(700,-600,1500) #monto inicial invertido en divisas

#CETES
base="RiesgosFinancieros/2022-2/Tarea/tasa_guber.txt"
plazos_cete= cbind(180,0)
contratos_cete= cbind(150,0)
nominal_cete= 10 

#BONOS M
btasadesc="RiesgosFinancieros/2022-2/Tarea/tasa_yield.txt"
tfcupon=cbind(0.065,0) #Tasa fija del cupón
plazos=cbind(3600,0) #Vencimiento del bono
plazocupon=cbind(182,0) #plazos fijos de cada cupón
contratos=cbind(10,0) #posición invertida
nominal=100
# itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada

#BONDES D
btasadescst="RiesgosFinancieros/2022-2/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2022-2/Tarea/tfondeo.txt"
plazos_bdm=cbind( 707,	707) #Vencimiento del bono
plazocupon_bdm=cbind( 28,	28) #plazos_bdm fijos de cada cupón
contratos_bdm=cbind(-1000, 0) #posición invertida
nominal_bdm=100

#FUTUROS TDC
bext="RiesgosFinancieros/2022-2/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2022-2/Tarea/tasa_fwd.txt"
SymbolsFX_ftdc<-cbind("USDMXN=X", "GBPUSD=X" ) 
plazos_fwd=cbind(5, 5)
contratos_fwd=cbind(100, 0)
kst_fwd=cbind(20.83, 0)
nominal_fwd=1
yext=1 #1 si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FUTUROS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2022-2/Tarea/tasa_guber.txt"
SymbolsEQ_find<-c("^MXX",  "GCARSOA1.MX") 
plazos_fwd_ind=cbind(53,53)
contratos_fwd_ind=cbind(50,0)
kst_fwd_ind=c(49525, 49525)
nominal_fwd_ind=1

#SWAP 
btasadesc_sw="RiesgosFinancieros/2022-2/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2022-2/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw= cbind(0.06,0.059) #se establece la tasa fija a pagar para cada swap
plazos_sw= cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw= cbind(28,28)#se establece el número de días que se pagará cada cupón
contratos_sw= cbind(16000000,-12000000) #se establece el número de contratos_sw de cada swap
nominal_sw= cbind(1,1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

#OPCIONES 
btasadesc_oir="RiesgosFinancieros/2022-2/Tarea/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2022-2/Tarea/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2022-2/Tarea/tvoltiie_opc.txt"
plazos_oir=cbind(1700, 700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor)
K_oir=cbind(0.058,0.06)
contratos_oir=cbind(1000,500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple

In [262]:
%%R
btasacupvar_sw

[1] "RiesgosFinancieros/2022-2/Tarea/tasa_DIRS_SW_OP.txt"


# Carga de datos

In [263]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_fx),t(pos_eq))

#Creación del objeto para guardar los datos
dataEnv<-new.env()
dataEnvFX<-new.env()

#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,to=fval)
getSymbols.yahoo(SymbolsFX,env=dataEnvFX,to=fval)

#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(dataEnv,align='remove.na',fill.gaps = T)
bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
#muestra de datos
#head(dataEnv$prices)
#head(dataEnvFX$prices)

#Nos quedamos con los precios de cierre
stock_prices = dataEnv$prices
stock_pricesFX=dataEnvFX$prices

#cambiar todo a pesos mexicanos
stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")

#Precios actuales
x0=stock_prices_EQFX[nrow(stock_prices_EQFX),]
#x0

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))
tail(aux2)

         Date EURUSD.X GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1: 2022-03-23 22.35971 26.89405 20.27220   20.73       73.71     77.99
2: 2022-03-24 22.26207 26.71136 20.22664   20.91       75.61     78.56
3: 2022-03-25 22.09215 26.47899 20.06630   21.00       75.77     78.59
4: 2022-03-28 21.99689 26.39227 20.03015   20.72       75.81     79.83
5: 2022-03-29 22.08800 26.32673 20.09125   20.70       74.29     78.58
6: 2022-03-30 22.14488 26.14256 19.96560   20.75       73.53     80.34


In [264]:
%%R
#CETES 

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)

#x_orig_gov=data.frame(data[2:n,1:m_gov])

x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)

nodos_gov=data.frame(data[1,2:m_gov])

In [265]:
%%R
#BONO M
#carga de tasas de descuento
data1<-read.table(btasadesc)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
nodos=data.frame(data1[1,2:m_orig])
n=n-1

m=ncol(plazos)
X=matrix(0,n-1,m)
for (i in 1:(n-1))
{
X[i,]=if(itpl==0){approx(nodos,X_orig[i,],plazos)$y}else{talamb(nodos,X_orig[i,],plazos)}
}
head(X)

           [,1] [,2]
[1,] 0.07537882   NA
[2,] 0.07631780   NA
[3,] 0.07698505   NA
[4,] 0.07454305   NA
[5,] 0.07670660   NA
[6,] 0.07720393   NA


In [266]:
%%R
#BONDE D
#carga de datos
#carga de tasas de descuento
data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
##X_orig_bd=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
#X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
#nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
print(head(X2_orig_bd))
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
print(head(X2_orig_bd))
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.

          V1         V2         V3         V4
2 2022-03-31 0.07845614 0.07749243 0.07650381
3 2022-03-30 0.07663158 0.07749243 0.07650381
4 2022-03-29 0.07663158 0.07749243 0.07685555
5 2022-03-28 0.07753656 0.07749243 0.07685555
6 2022-03-25 0.07663158 0.07749243 0.07685555
7 2022-03-24 0.07663158 0.07742413 0.07685555
        V1    V2
2 19981103 30.75
3 19981104  29.2
4 19981105  29.8
5 19981106  31.3
6 19981109  32.9
7 19981110 34.54
           V1   V2       Date   ID      fecha
1: 2022-03-31 6.52 2022-03-31 8550 2022-03-31
2: 2022-03-30 6.48 2022-03-30 8549 2022-03-30
3: 2022-03-29 6.46 2022-03-29 8548 2022-03-29
4: 2022-03-28 6.45 2022-03-28 8547 2022-03-28
5: 2022-03-25 6.47 2022-03-27 8546 2022-03-27
6: 2022-03-25 6.47 2022-03-26 8545 2022-03-26


In [267]:
%%R
   #CARGA DE DATOS PARA SWAP
  data1<-read.table(btasadesc_sw)
  n1<-nrow(data1)
  m1_orig_sw=ncol(data1)
  X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X1_orig_sw=X1_orig_sw%>%select(-V1)
  nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
  data2<-read.table(btasacupvar_sw)
  n2<-nrow(data2)
  m2_orig_sw=ncol(data2)
  X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X2_orig_sw=X2_orig_sw%>%select(-V1)
  nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  X2_orig_sw

           V2       V3       V4       V5       V6       V7       V8       V9
  1: 6.605831 6.689449 6.775184 6.828315 6.896106 6.905513 6.898793 6.784002
  2: 6.651778 6.735977 6.822308 6.875809 6.944071 6.946175 6.931947 6.795927
  3: 6.633530 6.717499 6.803593 6.856946 6.925022 6.934019 6.926814 6.763071
  4: 6.615352 6.699091 6.784949 6.838156 6.906045 6.944522 6.967215 6.804032
  5: 6.597443 6.680955 6.766581 6.819644 6.887349 6.932852 6.962704 6.771289
 ---                                                                        
251: 4.116708 4.168818 4.222248 4.255358 4.297605 4.264310 4.220471 4.106565
252: 4.120777 4.172939 4.226421 4.259565 4.301853 4.259305 4.206085 4.126408
253: 4.146701 4.199191 4.253009 4.286361 4.328916 4.299702 4.259921 4.202170
254: 4.141027 4.193445 4.247190 4.280496 4.322992 4.275557 4.217339 4.196554
255: 4.181767 4.234700 4.288974 4.322608 4.365523 4.336305 4.296433 4.258407
          V10      V11      V12      V13      V14      V15      V16      V17

In [268]:
  %%R
  #CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])
  x3_orig_oir

            V2        V3        V4         V5        V6        V7        V8
  1: 0.4665709 0.3328868 0.2742244 0.20591512 0.2510428 0.2783824 0.2798213
  2: 0.4905821 0.3499227 0.2891028 0.21831964 0.2634969 0.2793499 0.2967162
  3: 0.4905484 0.3498969 0.2890889 0.21831964 0.2634969 0.2793499 0.2967162
  4: 0.4904293 0.3498066 0.2890363 0.21831964 0.2634969 0.2793499 0.2967162
  5: 0.5088287 0.3620960 0.3017326 0.23280813 0.2869167 0.2902659 0.2967162
 ---                                                                       
251: 0.1647192 0.1137882 0.1019463 0.09472095 0.1607378 0.1796671 0.1929648
252: 0.1771317 0.1236374 0.1078410 0.09427439 0.1483829 0.1662206 0.1880526
253: 0.1740137 0.1215733 0.1057074 0.09179348 0.1438925 0.1627474 0.1850755
254: 0.1916142 0.1352480 0.1139142 0.09179348 0.1339689 0.1627474 0.1696939
255: 0.1932923 0.1365530 0.1146992 0.09179348 0.1329765 0.1627474 0.1746557
            V9       Date
  1: 0.2724996 2022-03-31
  2: 0.3224502 2022-03-30
  3: 0.322

In [269]:
%%R
#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))

x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)

###Para Dolar

if (yext==1)
{

#Creación del objeto para guardar los datos
  dataEnvFX_ftdc<-new.env()
  
#obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX_ftdc, to=fval)
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX_ftdc,align='remove.na',fill.gaps=T)
  
#Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_ftdc$prices[,2])),coredata(dataEnvFX_ftdc$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}
X3_ftdc

            Date USDMXN=X
   1: 2007-01-01 10.80300
   2: 2007-01-02 10.76050
   3: 2007-01-03 10.83000
   4: 2007-01-04 10.86500
   5: 2007-01-05 10.91200
  ---                    
3975: 2022-03-25 20.06630
3976: 2022-03-28 20.03015
3977: 2022-03-29 20.09125
3978: 2022-03-30 19.96560
3979: 2022-03-31 19.87815


In [270]:
%%R
# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ, to=fval)
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))
  X3_find

            Date      MXX
   1: 2007-01-02 26664.45
   2: 2007-01-03 26619.37
   3: 2007-01-04 26566.28
   4: 2007-01-05 26135.60
   5: 2007-01-08 26281.64
  ---                    
3825: 2022-03-24 55829.86
3826: 2022-03-25 55436.05
3827: 2022-03-28 55685.14
3828: 2022-03-29 56111.21
3829: 2022-03-30 55814.99


# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [271]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  
#head(x_orig_gov)  
lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
nrow(lin_gub)
lin_gub_bmybdst=data.table(Date=as.Date(lin_gub[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
nrow(lin_gub_bmybdst)
#lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor

lin_gub_bmybdst=data.table(Date=as.Date(X3_ftdc[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
print(nrow(lin_gub_bmybdst))
lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
print(nrow(lin_gub_bmybdst_flib))
lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd
print(nrow(lin_gub_bmybdst_flibfwd))
lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index
print(nrow(lin_gub_bmybdst_flibfwdspind))


lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
print(nrow(lin_gub_bmybdst_flibfwdspind_swcup))
lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp))
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot
print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs))
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp))
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol
print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol))
print(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

print(unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol))
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
print(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

[1] 255
[1] 254
[1] 254
[1] 253
[1] 253
[1] 253
[1] 253
[1] 253
[1] 253
           Date
  1: 2022-03-30
  2: 2022-03-29
  3: 2022-03-28
  4: 2022-03-25
  5: 2022-03-24
 ---           
249: 2021-04-09
250: 2021-04-08
251: 2021-04-07
252: 2021-04-06
253: 2021-04-05
           Date
  1: 2022-03-30
  2: 2022-03-29
  3: 2022-03-28
  4: 2022-03-25
  5: 2022-03-24
 ---           
249: 2021-04-09
250: 2021-04-08
251: 2021-04-07
252: 2021-04-06
253: 2021-04-05
           Date
  1: 2022-03-30
  2: 2022-03-29
  3: 2022-03-28
  4: 2022-03-25
  5: 2022-03-24
 ---           
249: 2021-04-09
250: 2021-04-08
251: 2021-04-07
252: 2021-04-06
253: 2021-04-05


In [272]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos

#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X1_orig=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)
# X2_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
# X2_orig_bd=X2_orig_bd%>%select(-Date)
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)

#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#historia de swaps
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

# Pasos para medir el riesgo por el método de Simulación Histórica

Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Valoración al día actual

## Acciones y Divisas



In [273]:
%%R
#Divisas y acciones 
x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=pos*x0_acc_div

print(V0_acc_div)

   EURUSD.X  GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1: 15501.42 -15685.53  29948.4 -103750       73530     96408


## Bonos

In [274]:
%%R
#CETES

#paso para interpolar las tasas
n_cete <-nrow(x_orig_gov)
m=ncol(plazos_cete)
x=matrix(0,n_cete-1,m)

for (i in 1:(n_cete-1)){
x[i,]=approx(nodos_gov,x_orig_gov[i,],plazos_cete)$y
}

x0=x[1,]

#función
bonocupcero = function(i,t){
1/(1+i*t/360)
}

V0=as.matrix(bonocupcero(x0,plazos_cete))*contratos_cete*nominal_cete #Valor actual de cada bono

V0_cete =V0[1]
V0_cete

[1] 1442.687


In [275]:
# Bono M
%%R
bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
x=x0  
N=as.integer(plazos/plazocupon)+1   
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0=X[1,] #tasas de descuento valor actual

bonoMyield(x0,plazos, plazocupon, tfcupon, nominal, contratos)

         [,1] [,2]
[1,] 935.1361   NA


In [276]:
%%R
#BONDE D CÁLCULO

X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)

N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo

plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos

#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

itpl=0
for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}
               else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))


In [277]:
%%R
bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m)){
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m)){
    if (j==1){
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else{
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N))){
    if(j==1){
      V0[j]=sum(V0f[j:N[j]])}
    else
    {V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])}}
  V0
  } 

V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)

V0_bd

          [,1] [,2]
[1,] -94292.17    0


## Forwards de Tipo de Cambio

In [278]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO VALORACIÓN

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)


for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)


X_futtdc=cbind(X1_fwtdc,X2_fwtdc,X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd
# V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd
V0_fwtdc

          [,1] [,2]
[1,] -85.61414    0


## Forward de indices

In [279]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos
m_ind=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m_ind) #DIVIDENDOS
X2_fwind=matrix(0,n,m_ind)

for (j in 1:n)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_find=as.matrix(X3_find)
X_futind=cbind(X1_fwind,X2_fwind,matrix(X3_find,n,ncol(X1_fwind)))

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
V0_fwind

         [,1] [,2]
[1,] 342308.9    0


## Swaps


In [280]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)
V0_sw

         [,1]     [,2]
[1,] 135101.8 80734.57


##Opciones de Tasa de Interés

In [281]:
%%R

##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
V0_oir

         [,1]      [,2]
[1,] 1.002318 0.2479957


# Integración de factores y cálculo de riesgo 


In [282]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
n_if=matrix(0,7,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(X_bd_ext) #bonde
n_if[3]=ncol(X_sw) #swaps
n_if[4]=ncol(X_oir) #opciones tasa de interés
n_if[5]=ncol(X_futtdc) #Forwards de tipo de cambio
n_if[6]=ncol(X_futind) #Forwards de índices
n_if[7]=ncol(x)-1 #cetes

#valor del portafolios

V0_port=cbind(V0_acc_div, V0_bd, V0_sw, V0_oir,V0_fwtdc,V0_fwind, V0_cete)
V0T_port=sum(V0_port)

#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,x_orig_gov, X_bd_ext,X_sw,X_oir,X_futtdc,X_futind, x[,1]) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
#alpha=0.98 #Nivel de Confianza para las medidas de riesgo

DeltaX_s=DeltaX_port
# print(head(DeltaX_s))

print(n_if)


     [,1]
[1,]    6
[2,]  156
[3,]   64
[4,]    6
[5,]    5
[6,]    6
[7,]    1


In [283]:
%%R

#Medidas de riesgo CON alisado

#Se necesita definir
#1) El valor del peso inicial del primer escenario "w0"
#2) La función de cuantil con vector de probabilidades no iguales
#3) La función de CVaR con probabilidades no iguales

#w0=0.05

#Creación de dos funciones que sirven para este fin
# Percentil con pesos de probabilidades
# 
# v  un vector de observaciones
# w Un vector numérico de valores positivos, en general es la distrubición.
# p  el valor de la probabilidad entre 0 y 1.
# 
# Esta función no interpola

wquantile <- function(v,w=rep(1,length(v)),p) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 


#esc_cvar=which(cumsum(p_esc[order(PLT[,1])])<pdca)


#p_esc[esc_cvar]

#tshs=cbind(PLT,p_esc)

w0=0.05
n=Ns
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root
lambda

#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}

p_esc=genera_esc(lambda,w0,n)
head(p_esc)

           [,1]
[1,] 0.05000000
[2,] 0.04750001
[3,] 0.04512501
[4,] 0.04286877
[5,] 0.04072533
[6,] 0.03868907


# Medición de Riesgo

## Acciones y divisas

In [284]:
%%R
#Medición de riesgo por instrumento, instrumento-factor de riesgo, instrumento - total


#Cálculo de matriz de pérdidas y ganancias Acciones y Divisas

#riesgo del acciones y divisas
m_fx= ncol(stock_pricesFX)
m_acc= ncol(stock_prices)
m=m_fx+m_acc    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_fx)
Vfr2_acc_div=matrix(0,Ns,m_acc)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_fx)
PGfr2_acc_div=matrix(0,Ns,m_acc)
PGT_acc_div=matrix(0,Ns,1) #Suma de las pérdidas y ganancias de cada instrumento
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

DeltaX_s_acc_div=DeltaX_s[,(1:n_if[1])]    #PASO CLAVE
x0_acc_div=stock_prices_EQFX[1,]   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*exp(DeltaX_s_acc_div[i,]))
  #PASO CLAVE
  V_acc_div[i,]=cbind(t(pos_fx),t(pos_eq))*X_s_acc_div[i,]
  #PASO CLAVE
  Vfr1_acc_div[i,]=t(pos_fx)*X_s_acc_div[i,1:m_fx]
  #PASO CLAVE
  Vfr2_acc_div[i,]=t(pos_eq)*X_s_acc_div[i,(m_fx+1):(m_fx+m_acc)]
  #PASO CLAVE
  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)
  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_fx])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_fx+1):(m_fx+m_acc)])
  PGT_acc_div[i,]=sum(PG_acc_div[i,])
  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}

#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_fx)
VaRfr2_acc_div=matrix(0,1,m_acc)
CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_fx)
CVaRfr2_acc_div=matrix(0,1,m_acc)

VaRCont_CA_acc_div=matrix(0,1,m)
VaRfr1_CA_acc_div=matrix(0,1,m_fx)
VaRfr2_CA_acc_div=matrix(0,1,m_acc)
CVaRCont_CA_acc_div=matrix(0,1,m)
CVaRfr1_CA_acc_div=matrix(0,1,m_fx)
CVaRfr2_CA_acc_div=matrix(0,1,m_acc)


for (i in (1:m))
{
  #Sin alisado
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])

  #Con alisado
  VaRCont_CA_acc_div[i]=wquantile(PG_acc_div[,i],p_esc, 1-alpha)
  CVaRCont_CA_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_CA_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])

  if (i<=m_fx)
  {
  VaRfr1_acc_div[i]=equantile(PGfr1_acc_div[,i],1-alpha,Ns)
  CVaRfr1_acc_div[i]= mean(merge(which(PGfr1_acc_div[,i]<VaRfr1_acc_div[i]),cbind(seq(1,Ns),PGfr1_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
  if (i<=m_acc)
  {  
  VaRfr2_acc_div[i]=equantile(PGfr2_acc_div[,i],1-alpha,Ns)
  CVaRfr2_acc_div[i]= mean(merge(which(PGfr2_acc_div[,i]<VaRfr2_acc_div[i]),cbind(seq(1,Ns),PGfr2_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
}


#VaR Total
VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])

#VaR y CVaR Total por factor de riesgo 1
#Sin alisado
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])

#Con alisado
VaRTotalfr1_CA_acc_div = wquantile(PGfr1T_acc_div,p_esc, 1-alpha)
CVaRTotalfr1_CA_acc_div = mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_CA_acc_div),])

#VaR y CVaR Total por factor de riesgo 2
#Sin alisado
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])

#Con alisado
VaRTotalfr2_CA_acc_div = wquantile(PGfr2T_acc_div,p_esc, 1-alpha)
CVaRTotalfr2_CA_acc_div = mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_CA_acc_div),])

print(cbind(VaRTotal_acc_div,sum(V0_acc_div), VaRCont_acc_div, V0_acc_div))
cbind(CVaRTotal_acc_div,sum(V0_acc_div), CVaRCont_acc_div, V0_acc_div)
cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div)
cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div)

   VaRTotal_acc_div       V2        V1        V2        V3        V4        V5
1:        -4306.785 95952.28 -149.0083 -160.1994 -317.2778 -2940.283 -2955.189
          V6 EURUSD.X  GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1: -2616.769 15501.42 -15685.53  29948.4 -103750       73530     96408
     CVaRTotal_acc_div CVaRTotalfr1_acc_div CVaRTotalfr2_acc_div
[1,]         -5307.676             -423.176            -5309.633


In [285]:
%%R
#VaR y CVaR  Divisas
Acc_VaR_CVaR <- cbind(VaRCont_acc_div[,1:3], 
CVaRCont_acc_div[,1:3], VaRCont_CA_acc_div[,1:3],CVaRCont_CA_acc_div[,1:3] )
colnames(Acc_VaR_CVaR) <- c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")

Acc_VaR_CVaR <- rbind(Acc_VaR_CVaR, c(VaRTotalfr1_acc_div, CVaRTotalfr1_acc_div, VaRTotalfr1_CA_acc_div, CVaRTotalfr1_CA_acc_div))
rownames(Acc_VaR_CVaR) <- c(names(stock_pricesFX), "Total")

Acc_VaR_CVaR


            VaR SA   CVaR SA    VaR CA   CVaR CA
EURUSD.X -149.0083 -189.9811 -214.0865 -271.4772
GBPUSD.X -160.1994 -179.9433 -130.5663 -158.3959
USDMXN.X -317.2778 -442.8901 -640.9600       NaN
Total    -357.8263 -423.1760 -493.7925       NaN


In [286]:
%%R
#VaR y CVaR Acciones
Div_VaR_CVaR <- cbind(VaRCont_acc_div[,4:6], CVaRCont_acc_div[,4:6], VaRCont_CA_acc_div[,4:6],CVaRCont_CA_acc_div[,4:6] )
colnames(Div_VaR_CVaR) <- c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")

Div_VaR_CVaR <- rbind(Div_VaR_CVaR, c(VaRTotalfr2_acc_div, CVaRTotalfr2_acc_div, VaRTotalfr2_CA_acc_div, CVaRTotalfr2_CA_acc_div))
rownames(Div_VaR_CVaR) <- c(names(stock_prices), "Total")

Div_VaR_CVaR

               VaR SA   CVaR SA    VaR CA   CVaR CA
AMXL.MX     -2940.283 -4142.288 -3426.218 -4736.788
GCARSOA1.MX -2955.189 -3435.131 -2592.736 -3068.185
WALMEX.MX   -2616.769 -3111.195 -3115.921 -3269.536
Total       -4559.199 -5309.633 -4888.826 -5682.427


In [287]:
%%R
# VaR y CVaR TOTAL 
print("VaR Total Acciones y Divisas:")
print(VaRTotal_acc_div)

print('CVaR Total Acciones y divisas:')
print(CVaRTotal_acc_div)

[1] "VaR Total Acciones y Divisas:"
[1] -4306.785
[1] "CVaR Total Acciones y divisas:"
[1] -5307.676


## Riesgo de CETES

In [288]:
%%R
#CETES

#Cálculo de matriz de pérdidas y ganancias 
#dimensión
m=n_if[7]  #PASO CLAVE
X_s_cete= matrix(0,Ns,m)    #PASO CLAVE
PG_cete= matrix(0,Ns,m)
V_cete = matrix(0,Ns,m)

DeltaX_s_cete=DeltaX_s[,ncol(DeltaX_s)]    #PASO CLAVE
x0_cete=x[1,1] #Escenario base

for (i in 1:Ns)
{
  X_s_cete[i]=x0_cete*exp(DeltaX_s_cete[i])
  V_cete[i] = bonocupcero(X_s_cete[i],plazos_cete)*contratos_cete*nominal_cete
  PG_cete[i]=V_cete[i]-V0_cete
}

# VaR y CVaR sin alisado
VaR_cetes_SA=equantile(PG_cete,1-alpha,Ns)
CVaR_cetes_SA= mean(PG_cete[which(PG_cete<VaR_cetes_SA),])

# VaR y CVaR con alisado

VaR_cetes_CA=wquantile(PG_cete,p_esc,1-alpha)
CVaR_cetes_CA= mean(PG_cete[which(PG_cete<VaR_cetes_CA),])

#Resumen
Cetes_VaR_CVaR <- cbind(VaR_cetes_SA, CVaR_cetes_SA, VaR_cetes_CA, CVaR_cetes_CA)
colnames(Cetes_VaR_CVaR) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(Cetes_VaR_CVaR) <- c("CETES")
Cetes_VaR_CVaR

         VaR SA  CVaR SA     VaR CA   CVaR CA
CETES -1.054782 -1.83657 -0.6318718 -1.346447


## Riesgo de Bondes D

In [289]:
%%R
#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(N_bd)    #PASO CLAVE
X_s_bd=matrix(0,Ns,n_if[2]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_bd=matrix(0,Ns,m)
Vfr1_bd=matrix(0,Ns,m)
Vfr2_bd=matrix(0,Ns,m)
Vfr3_bd=matrix(0,Ns,m)
PG_bd=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_bd=matrix(0,Ns,m)
PGfr2_bd=matrix(0,Ns,m)
PGfr3_bd=matrix(0,Ns,m)
PGT_bd=matrix(0,Ns,1)
PGfr1T_bd=matrix(0,Ns,1)
PGfr2T_bd=matrix(0,Ns,1)
PGfr3T_bd=matrix(0,Ns,1)

DeltaX_s_bd=DeltaX_s[,sum(n_if[1:1],1):sum(n_if[1:2])]    #PASO CLAVE
x0_bd=X_bd_ext[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_bd[i,]=x0_bd*exp(DeltaX_s_bd[i,])
  #PASO CLAVE
  V_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr1_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr2_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr3_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  PG_bd[i,]=V_bd[i,]-V0_bd
  PGfr1_bd[i,]=Vfr1_bd[i,]-V0_bd
  PGfr2_bd[i,]=Vfr2_bd[i,]-V0_bd
  PGfr3_bd[i,]=Vfr3_bd[i,]-V0_bd
  PGT_bd[i,]=sum(PG_bd[i,])
  PGfr1T_bd[i,]=sum(PGfr1_bd[i,])
  PGfr2T_bd[i,]=sum(PGfr2_bd[i,])
  PGfr3T_bd[i,]=sum(PGfr3_bd[i,])
}


#VaR y CVaR Total

#Sin alisado
VaRTotal_bd=equantile(PGT_bd,1-alpha,Ns)
CVaRTotal_bd= mean(merge(which(PGT_bd<VaRTotal_bd),cbind(seq(1,Ns),PGT_bd), by.x=1,by.y=1)[,2])

VaRTotalfr1_bd=equantile(PGfr1T_bd,1-alpha,Ns)
CVaRTotalfr1_bd= mean(PGfr1T_bd[which(PGfr1T_bd<VaRTotalfr1_bd),])

VaRTotalfr2_bd=equantile(PGfr2T_bd,1-alpha,Ns)
CVaRTotalfr2_bd= mean(PGfr2T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])

VaRTotalfr3_bd=equantile(PGfr3T_bd,1-alpha,Ns)
CVaRTotalfr3_bd= mean(PGfr3T_bd[which(PGfr2T_bd<VaRTotalfr3_bd),])

#Con alisado
VaRTotal_CA_bd=wquantile(PGT_bd,p_esc,1-alpha)
CVaRTotal_CA_bd= mean(merge(which(PGT_bd<VaRTotal_CA_bd),cbind(seq(1,Ns),PGT_bd), by.x=1,by.y=1)[,2])

VaRTotalfr1_CA_bd=wquantile(PGfr1T_bd,p_esc,1-alpha)
CVaRTotalfr1_CA_bd= mean(PGfr1T_bd[which(PGfr1T_bd<VaRTotalfr1_CA_bd),])

VaRTotalfr2_CA_bd=wquantile(PGfr2T_bd,p_esc,1-alpha)
CVaRTotalfr2_CA_bd= mean(PGfr2T_bd[which(PGfr2T_bd<VaRTotalfr2_CA_bd),])

VaRTotalfr3_CA_bd=wquantile(PGfr3T_bd,p_esc,1-alpha)
CVaRTotalfr3_CA_bd= mean(PGfr3T_bd[which(PGfr2T_bd<VaRTotalfr3_CA_bd),])


#Resumen
Bonde_VaR_CVaR <- rbind(c(VaRTotal_bd,CVaRTotal_bd, VaRTotal_CA_bd, CVaRTotal_CA_bd),
                        c(VaRTotalfr1_bd,CVaRTotalfr1_bd,VaRTotalfr1_CA_bd,CVaRTotalfr1_CA_bd ),
                        c(VaRTotalfr2_bd, CVaRTotalfr2_bd, VaRTotalfr2_CA_bd, CVaRTotalfr2_CA_bd),
                        c(VaRTotalfr3_bd,CVaRTotalfr3_bd,VaRTotalfr3_CA_bd,CVaRTotalfr3_CA_bd ))
colnames(Bonde_VaR_CVaR ) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(Bonde_VaR_CVaR ) <- c("Total BONDE", "FR1", "FR2", "FR3")
Bonde_VaR_CVaR 

              VaR SA  CVaR SA   VaR CA  CVaR CA
Total BONDE 2538.386 2440.554 2581.671 2514.286
FR1         2665.051 2619.449 2659.151 2579.093
FR2         2689.913 2621.877 2813.469 2736.874
FR3         2780.048 2859.192 2773.408 2859.192


## Riesgo SWAPS

In [290]:
%%R
#Cálculo de matriz de pérdidas y ganancias SWAP

#riesgo del swap
m=ncol(N)    #PASO CLAVE
X_s_sw=matrix(0,Ns,n_if[3]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_sw=matrix(0,Ns,m)
Vfr1_sw=matrix(0,Ns,m)
Vfr2_sw=matrix(0,Ns,m)
PG_sw=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_sw=matrix(0,Ns,m)
PGfr2_sw=matrix(0,Ns,m)
PGT_sw=matrix(0,Ns,1)
PGfr1T_sw=matrix(0,Ns,1)
PGfr2T_sw=matrix(0,Ns,1)

DeltaX_s_sw=DeltaX_s[,sum(n_if[1:2],1):sum(n_if[1:3])]    #PASO CLAVE
x0_sw=as.numeric(c(XtfwdT[1,],Xvp[1,]))   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_sw[i,]=x0_sw*exp(DeltaX_s_sw[i,])
  #PASO CLAVE
  V_sw[i,]=swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  Vfr1_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,],N)
  #PASO CLAVE
  Vfr2_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  PG_sw[i,]=V_sw[i,]-V0_sw
  PGfr1_sw[i,]=Vfr1_sw[i,]-V0_sw
  PGfr2_sw[i,]=Vfr2_sw[i,]-V0_sw
  PGT_sw[i,]=sum(PG_sw[i,])
  PGfr1T_sw[i,]=sum(PGfr1_sw[i,])
  PGfr2T_sw[i,]=sum(PGfr2_sw[i,])
}

#VaR por posición 
VaRCont_sw=matrix(0,1,m)
VaRfr1_sw=matrix(0,1,m)
VaRfr2_sw=matrix(0,1,m)

CVaRCont_sw=matrix(0,1,m)
CVaRfr1_sw=matrix(0,1,m)
CVaRfr2_sw=matrix(0,1,m)

VaRCont_CA_sw=matrix(0,1,m)
VaRfr1_CA_sw=matrix(0,1,m)
VaRfr2_CA_sw=matrix(0,1,m)

CVaRfr1_CA_sw=matrix(0,1,m)
CVaRfr2_CA_sw=matrix(0,1,m)
CVaRCont_CA_sw=matrix(0,1,m)

for (i in (1:m))
{
  #Sin alisado
  VaRCont_sw[i]=equantile(PG_sw[,i],1-alpha,Ns)
  VaRfr1_sw[i]=equantile(PGfr1_sw[,i],1-alpha,Ns)
  VaRfr2_sw[i]=equantile(PGfr2_sw[,i],1-alpha,Ns)

  CVaRfr1_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])

  #Con alisado
  VaRCont_CA_sw[i]=wquantile(PG_sw[,i],p_esc, 1-alpha)
  VaRfr1_CA_sw[i]=wquantile(PGfr1_sw[,i],p_esc,1-alpha)
  VaRfr2_CA_sw[i]=wquantile(PGfr2_sw[,i],p_esc,1-alpha)

  CVaRfr1_CA_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_CA_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_CA_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_CA_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_CA_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_CA_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])
}

#Resumen VaR y CVaR posición Larga
SwapL_VaR_CVaR <- rbind(c(VaRCont_sw[,1], CVaRCont_sw[,1], VaRCont_CA_sw[,1], CVaRCont_CA_sw[,1]), 
                        c(VaRfr1_sw[,1],CVaRfr1_sw[,1],VaRfr1_CA_sw[,1],CVaRfr1_CA_sw[,1]), 
                        c(VaRfr2_sw[,1],CVaRfr2_sw[,1],VaRfr2_CA_sw[,1],CVaRfr2_CA_sw[,1]))
colnames(SwapL_VaR_CVaR) <- c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(SwapL_VaR_CVaR) <- c("Total Swap Largo", "FR1", "FR2")
SwapL_VaR_CVaR

                      VaR SA     CVaR SA      VaR CA     CVaR CA
Total Swap Largo -22715.7154 -31329.1950 -28809.6013 -39513.6686
FR1              -22754.7864 -31329.4430 -28814.4369 -39513.3639
FR2                -191.8218   -318.7339   -305.0335   -399.3062


In [291]:
%%R
#Resumen VaR y CVaR posición Corta
SwapC_VaR_CVaR <- rbind(c(VaRCont_sw[,2], CVaRCont_sw[,2], VaRCont_CA_sw[,2], CVaRCont_CA_sw[,2]), 
                        c(VaRfr1_sw[,2],CVaRfr1_sw[,2],VaRfr1_CA_sw[,2],CVaRfr1_CA_sw[,2]), 
                        c(VaRfr2_sw[,2],CVaRfr2_sw[,2],VaRfr2_CA_sw[,2],CVaRfr2_CA_sw[,2]))
colnames(SwapC_VaR_CVaR) <- c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(SwapC_VaR_CVaR) <- c("Total Swap Corto", "FR1", "FR2")
SwapC_VaR_CVaR


                      VaR SA     CVaR SA    VaR CA    CVaR CA
Total Swap Corto -11530.4241 -14592.2087 -3485.368 -6764.8718
FR1              -11526.1113 -14612.1535 -3492.049 -6775.5557
FR2                 -93.1214   -138.7824  -133.843  -176.7767


In [292]:
%%R
#VaR y CVaR Total

#Sin alisado
VaRTotal_sw=equantile(PGT_sw,1-alpha,Ns)
CVaRTotal_sw= mean(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])

VaRTotalfr1_sw=equantile(PGfr1T_sw,1-alpha,Ns)
CVaRTotalfr1_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])

VaRTotalfr2_sw=equantile(PGfr2T_sw,1-alpha,Ns)
CVaRTotalfr2_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])

#Con alisado
VaRTotal_CA_sw=wquantile(PGT_sw,p_esc,1-alpha)
CVaRTotal_CA_sw= mean(merge(which(PGT_sw<VaRTotal_CA_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])

VaRTotalfr1_CA_sw=wquantile(PGfr1T_sw,p_esc,1-alpha)
CVaRTotalfr1_CA_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_CA_sw),])

VaRTotalfr2_CA_sw=wquantile(PGfr2T_sw,p_esc,1-alpha)
CVaRTotalfr2_CA_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_CA_sw),])

#Resumen Total VaR y CVaR
Swaps_VaR_CVaR <- rbind(c(VaRTotal_sw,CVaRTotal_sw,VaRTotal_CA_sw,CVaRTotal_CA_sw), 
                        c(VaRTotalfr1_sw,CVaRTotalfr1_sw,VaRTotalfr1_CA_sw,CVaRTotalfr1_CA_sw), 
                        c(VaRTotalfr2_sw,CVaRTotalfr2_sw,VaRTotalfr2_CA_sw,CVaRTotalfr2_CA_sw))
colnames(Swaps_VaR_CVaR) <- c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(Swaps_VaR_CVaR) <- c("Total Swaps", "FR1", "FR2")
Swaps_VaR_CVaR

# print(cbind(VaRTotal_sw,sum(V0_sw), VaRCont_sw, V0_sw))
# print(cbind(CVaRTotal_sw,sum(V0_sw), CVaRCont_sw, V0_sw))
# print(cbind(VaRTotal_sw,VaRTotalfr1_sw,VaRTotalfr2_sw))
# print(cbind(CVaRTotal_sw,CVaRTotalfr1_sw,CVaRTotalfr2_sw))

                 VaR SA     CVaR SA      VaR CA     CVaR CA
Total Swaps -20867.4743 -30389.8207 -28031.8623 -37973.0833
FR1         -20890.8051 -30406.5182 -28031.0566 -37971.1288
FR2           -284.7163   -457.5163   -438.8765   -576.0829


## Opciones Tasa de Interés

In [293]:
%%R
#Cálculo de matriz de pérdidas y ganancias Opciones Tasa de interés
#dimensión
m=ncol(plazos_oir)  #PASO CLAVE
X_s_oir=matrix(0,Ns,n_if[4]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s[,sum(n_if[1:3],1):sum(n_if[1:4])]  #PASO CLAVE
x0_oir=X_oir[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*exp(DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_oir
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}
PG_oir[1:5,]
PGfr1_oir[1:5,]
PGfr2_oir[1:5,]
PGT_oir[1:5,]

#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)

VaRCont_CA_oir=matrix(0,1,m)
CVaRCont_CA_oir=matrix(0,1,m)
VaRfr1_CA_oir=matrix(0,1,m)
CVaRfr1_CA_oir=matrix(0,1,m)
VaRfr2_CA_oir=matrix(0,1,m)
CVaRfr2_CA_oir=matrix(0,1,m)
VaRfr3_CA_oir=matrix(0,1,m)
CVaRfr3_CA_oir=matrix(0,1,m)

for (i in (1:m))
{
  #Sin alisado
  VaRCont_oir[i]=equantile(PG_oir[,i],1-alpha,Ns)
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])

  VaRfr1_oir[i]=equantile(PGfr1_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])

  VaRfr2_oir[i]=equantile(PGfr2_oir[,i],1-alpha,Ns)
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])

  VaRfr3_oir[i]=equantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])

  #Con alisado
  VaRCont_CA_oir[i]=wquantile(PG_oir[,i],p_esc,1-alpha)
  CVaRCont_CA_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_CA_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])

  VaRfr1_CA_oir[i]=wquantile(PGfr1_oir[,i],p_esc,1-alpha)
  CVaRfr1_CA_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_CA_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])

  VaRfr2_CA_oir[i]=wquantile(PGfr2_oir[,i],p_esc,1-alpha)
  CVaRfr2_CA_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_CA_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])

  VaRfr3_CA_oir[i]=wquantile(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRfr3_CA_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_CA_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
}

#Resumen
Opcion1_VaR_CVaR <- rbind(c(VaRCont_oir[1],CVaRCont_oir[1], VaRCont_CA_oir[1], CVaRCont_CA_oir[1]),
                          c(VaRfr1_oir[1],CVaRfr1_oir[1], VaRfr1_CA_oir[1], CVaRfr1_CA_oir[1]),
                          c(VaRfr2_oir[1], CVaRfr2_oir[1], VaRfr2_CA_oir[1], CVaRfr2_CA_oir[1]),
                          c(VaRfr3_oir[1], CVaRfr3_oir[1], VaRfr3_CA_oir[1], CVaRfr3_CA_oir[1])
)
                  
colnames(Opcion1_VaR_CVaR ) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(Opcion1_VaR_CVaR ) <- c("Total Opción 1", "FR1", "FR2", "FR3")
Opcion1_VaR_CVaR 


                    VaR SA     CVaR SA       VaR CA      CVaR CA
Total Opción 1 -0.03932438 -0.05134277 -0.049025948 -0.062577812
FR1            -0.01295580 -0.01772750 -0.016842984 -0.022258227
FR2            -0.06213690 -0.08351985 -0.079049530 -0.103885318
FR3            -0.01294514 -0.02175582 -0.005344718 -0.009218226


In [294]:
%%R
#Resumen Opción 2
Opcion2_VaR_CVaR <- rbind(c(VaRCont_oir[2],CVaRCont_oir[2], VaRCont_CA_oir[2], CVaRCont_CA_oir[2]),
                          c(VaRfr1_oir[2], CVaRfr1_oir[2], VaRfr1_CA_oir[2], CVaRfr1_CA_oir[2]),
                          c(VaRfr2_oir[2], CVaRfr2_oir[2], VaRfr2_CA_oir[2], CVaRfr2_CA_oir[2]),
                          c(VaRfr3_oir[2], CVaRfr3_oir[2], VaRfr3_CA_oir[2], CVaRfr3_CA_oir[2])
)
                  
colnames(Opcion2_VaR_CVaR ) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(Opcion2_VaR_CVaR ) <- c("Total Opción 2", "FR1", "FR2", "FR3")
Opcion2_VaR_CVaR 

                     VaR SA      CVaR SA       VaR CA      CVaR CA
Total Opción 2 -0.022634754 -0.037764922 -0.009295559 -0.015981896
FR1            -0.001503600 -0.002043248 -0.001923494 -0.002554849
FR2            -0.017948848 -0.030117192 -0.007270346 -0.012399739
FR3            -0.005236803 -0.008677545 -0.002224101 -0.003753300


In [295]:
%%R
#OPCIONES
#VaR Total Sin alisado
VaRTotal_oir=equantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])

VaRTotalfr1_oir=equantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])

VaRTotalfr2_oir=equantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])

VaRTotalfr3_oir=equantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])

#VaR Total Con alisado
VaRTotal_CA_oir=wquantile(PGT_oir,p_esc,1-alpha)
CVaRTotal_CA_oir= mean(merge(which(PGT_oir<VaRTotal_CA_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])

VaRTotalfr1_CA_oir=wquantile(PGfr1T_oir,p_esc,1-alpha)
CVaRTotalfr1_CA_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_CA_oir),])

VaRTotalfr2_CA_oir=wquantile(PGfr2T_oir,p_esc,1-alpha)
CVaRTotalfr2_CA_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_CA_oir),])

VaRTotalfr3_CA_oir=wquantile(PGfr3T_oir,p_esc,1-alpha)
CVaRTotalfr3_CA_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr3_CA_oir),])

#Resumen Opciones TOTAL 
OpcionT_VaR_CVaR <- rbind(c(VaRTotal_oir,CVaRTotal_oir, VaRTotal_CA_oir, CVaRTotal_CA_oir),
                          c(VaRTotalfr1_oir,CVaRTotalfr1_oir, VaRTotalfr1_CA_oir, CVaRTotalfr1_CA_oir),
                          c(VaRTotalfr2_oir,CVaRTotalfr2_oir, VaRTotalfr2_CA_oir, CVaRTotalfr2_CA_oir),
                          c(VaRTotalfr3_oir,CVaRTotalfr3_oir, VaRTotalfr3_CA_oir, CVaRTotalfr3_CA_oir)                        
)
                  
colnames(OpcionT_VaR_CVaR ) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(OpcionT_VaR_CVaR ) <- c("Total Opción 1", "FR1", "FR2", "FR3")
OpcionT_VaR_CVaR 


                     VaR SA     CVaR SA       VaR CA      CVaR CA
Total Opción 1 -0.006949365 -0.01156733 -0.004081966 -0.005904569
FR1            -0.014484939 -0.01976564 -0.018766478 -0.024813076
FR2            -0.009584753 -0.01031072 -0.010550415 -0.010680197
FR3            -0.018181942  0.06215037 -0.007568818  0.058108351


##Riesgo de Forwards TdC

In [296]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC
#dimensión
m=ncol(plazos_fwd)  #PASO CLAVE
X_s_fwtdc=matrix(0,Ns,n_if[5]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwtdc=matrix(0,Ns,m)
Vfr1_fwtdc=matrix(0,Ns,m)
Vfr2_fwtdc=matrix(0,Ns,m)
Vfr3_fwtdc=matrix(0,Ns,m)
PG_fwtdc=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwtdc=matrix(0,Ns,m)
PGfr2_fwtdc=matrix(0,Ns,m)
PGfr3_fwtdc=matrix(0,Ns,m)
PGT_fwtdc=matrix(0,Ns,1)
PGfr1T_fwtdc=matrix(0,Ns,1)
PGfr2T_fwtdc=matrix(0,Ns,1)
PGfr3T_fwtdc=matrix(0,Ns,1)

DeltaX_s_fwtdc=DeltaX_s[,sum(n_if[1:4],1):sum(n_if[1:5])]  #PASO CLAVE
x0_fwtdc=X_futtdc[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_fwtdc[i,]=x0_fwtdc*exp(DeltaX_s_fwtdc[i,])
  #PASO CLAVE
  V_fwtdc[i,]=futuroTC(plazos_fwd,X_s_fwtdc[i,1:((n_if[5]-1)/2)],X_s_fwtdc[i,((n_if[5]-1)/2+1):(n_if[5]-1)],X_s_fwtdc[i,(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd     
  #PASO CLAVE
  Vfr1_fwtdc[i,]=futuroTC(plazos_fwd,X_s_fwtdc[i,1:((n_if[5]-1)/2)],x0_fwtdc[((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd
  #PASO CLAVE
  Vfr2_fwtdc[i,]=futuroTC(plazos_fwd,x0_fwtdc[1:((n_if[5]-1)/2)],X_s_fwtdc[i,((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd 
  PG_fwtdc[i,]=V_fwtdc[i,]-V0_fwtdc
  PGfr1_fwtdc[i,]=Vfr1_fwtdc[i,]-V0_fwtdc
  PGfr2_fwtdc[i,]=Vfr2_fwtdc[i,]-V0_fwtdc
  PGfr3_fwtdc[i,]=Vfr3_fwtdc[i,]-V0_fwtdc
  PGT_fwtdc[i,]=sum(PG_fwtdc[i,])
  PGfr1T_fwtdc[i,]=sum(PGfr1_fwtdc[i,])
  PGfr2T_fwtdc[i,]=sum(PGfr2_fwtdc[i,])
  PGfr3T_fwtdc[i,]=sum(PGfr3_fwtdc[i,])
}

#VaR y CVaR Total

#Sin alisado
VaRTotal_fwtdc=equantile(PGT_fwtdc,1-alpha,Ns)
CVaRTotal_fwtdc= mean(merge(which(PGT_fwtdc<VaRTotal_fwtdc),cbind(seq(1,Ns),PGT_fwtdc), by.x=1,by.y=1)[,2])

VaRTotalfr1_fwtdc=equantile(PGfr1T_fwtdc,1-alpha,Ns)
CVaRTotalfr1_fwtdc= mean(PGfr1T_fwtdc[which(PGfr1T_fwtdc<VaRTotalfr1_fwtdc),])

VaRTotalfr2_fwtdc=equantile(PGfr2T_fwtdc,1-alpha,Ns)
CVaRTotalfr2_fwtdc= mean(PGfr2T_fwtdc[which(PGfr2T_fwtdc<VaRTotalfr2_fwtdc),])

VaRTotalfr3_fwtdc=equantile(PGfr3T_fwtdc,1-alpha,Ns)
CVaRTotalfr3_fwtdc= mean(PGfr3T_fwtdc[which(PGfr2T_fwtdc<VaRTotalfr3_fwtdc),])

#Con  alisado
VaRTotal_CA_fwtdc=wquantile(PGT_fwtdc,p_esc,1-alpha)
CVaRTotal_CA_fwtdc= mean(merge(which(PGT_fwtdc<VaRTotal_CA_fwtdc),cbind(seq(1,Ns),PGT_fwtdc), by.x=1,by.y=1)[,2])

VaRTotalfr1_CA_fwtdc=wquantile(PGfr1T_fwtdc,p_esc,1-alpha)
CVaRTotalfr1_CA_fwtdc= mean(PGfr1T_fwtdc[which(PGfr1T_fwtdc<VaRTotalfr1_CA_fwtdc),])

VaRTotalfr2_CA_fwtdc=wquantile(PGfr2T_fwtdc,p_esc,1-alpha)
CVaRTotalfr2_CA_fwtdc= mean(PGfr2T_fwtdc[which(PGfr2T_fwtdc<VaRTotalfr2_CA_fwtdc),])

VaRTotalfr3_CA_fwtdc=wquantile(PGfr3T_fwtdc,p_esc,1-alpha)
CVaRTotalfr3_CA_fwtdc= mean(PGfr3T_fwtdc[which(PGfr2T_fwtdc<VaRTotalfr3_CA_fwtdc),])

#Resumen
FWD_TDC_VaR_CVaR <- rbind(c(VaRTotal_fwtdc,CVaRTotal_fwtdc, VaRTotal_CA_fwtdc, CVaRTotal_CA_fwtdc),
                          c(VaRTotalfr1_fwtdc,CVaRTotalfr1_fwtdc,VaRTotalfr1_CA_fwtdc,CVaRTotalfr1_CA_fwtdc),
                          c(VaRTotalfr2_fwtdc, CVaRTotalfr2_fwtdc, VaRTotalfr2_CA_fwtdc, CVaRTotalfr2_CA_fwtdc),
                          c(VaRTotalfr3_fwtdc,CVaRTotalfr3_fwtdc,VaRTotalfr3_CA_fwtdc,CVaRTotalfr3_CA_fwtdc)
                          )
colnames(FWD_TDC_VaR_CVaR ) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(FWD_TDC_VaR_CVaR ) <- c("Total Forward TDC", "FR1", "FR2", "FR3")
FWD_TDC_VaR_CVaR

                        VaR SA      CVaR SA      VaR CA      CVaR CA
Total Forward TDC -15.89581486 -20.49460761 -7.90416412 -13.17497266
FR1                -0.05007073  -0.07011861 -0.06622219  -0.08864067
FR2                -0.03411734  -0.06210918 -0.01740035  -0.02824682
FR3                85.61414454  85.61414454 85.61414454  85.61414454


##Riesgo Forward Índice

In [297]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS IPC
m=ncol(plazos_fwd_ind)  #PASO CLAVE
X_s_fwind=matrix(0,Ns,n_if[6]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwind=matrix(0,Ns,m)
Vfr1_fwind=matrix(0,Ns,m)
Vfr2_fwind=matrix(0,Ns,m)
Vfr3_fwind=matrix(0,Ns,m)
PG_fwind=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwind=matrix(0,Ns,m)
PGfr2_fwind=matrix(0,Ns,m)
PGfr3_fwind=matrix(0,Ns,m)
PGT_fwind=matrix(0,Ns,1)
PGfr1T_fwind=matrix(0,Ns,1)
PGfr2T_fwind=matrix(0,Ns,1)
PGfr3T_fwind=matrix(0,Ns,1)

DeltaX_s_fwind=DeltaX_s[,sum(n_if[1:5],1):sum(n_if[1:6])]  #PASO CLAVE
x0_fwind=X_futind[1,] #PASO CLAVE


for (i in 1:Ns)
{
  X_s_fwind[i,]=x0_fwind* exp(DeltaX_s_fwind[i,])
  V_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind     
  
  Vfr1_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 

  Vfr2_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 

  Vfr3_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  
  
  PG_fwind[i,]=V_fwind[i,]-V0_fwind
  PGfr1_fwind[i,]=Vfr1_fwind[i,]-V0_fwind
  PGfr2_fwind[i,]=Vfr2_fwind[i,]-V0_fwind
  PGfr3_fwind[i,]=Vfr3_fwind[i,]-V0_fwind
  PGT_fwind[i,]=sum(PG_fwind[i,])
  PGfr1T_fwind[i,]=sum(PGfr1_fwind[i,])
  PGfr2T_fwind[i,]=sum(PGfr2_fwind[i,])
  PGfr3T_fwind[i,]=sum(PGfr3_fwind[i,])
}

#VaR por posición
VaRCont_fwind=matrix(0,1,m)
VaRfr1_fwind=matrix(0,1,m)
VaRfr2_fwind=matrix(0,1,m)
VaRfr3_fwind=matrix(0,1,m)
CVaRCont_fwind=matrix(0,1,m)
CVaRfr1_fwind=matrix(0,1,m)
CVaRfr2_fwind=matrix(0,1,m)
CVaRfr3_fwind=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_fwind[i]=quantile(PG_fwind[,i],1-alpha,Ns)
  VaRfr1_fwind[i]=quantile(PGfr1_fwind[,i],1-alpha,Ns)
  VaRfr2_fwind[i]=quantile(PGfr2_fwind[,i],1-alpha,Ns)
  VaRfr3_fwind[i]=quantile(PGfr3_fwind[,i],1-alpha,Ns)
  CVaRfr1_fwind[i]= mean(merge(which(PGfr1_fwind[,i]<VaRfr1_fwind[i]),cbind(seq(1,Ns),PGfr1_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_fwind[i]= mean(merge(which(PGfr2_fwind[,i]<VaRfr2_fwind[i]),cbind(seq(1,Ns),PGfr2_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_fwind[i]= mean(merge(which(PGfr3_fwind[,i]<VaRfr3_fwind[i]),cbind(seq(1,Ns),PGfr3_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_fwind[i]= mean(merge(which(PG_fwind[,i]<VaRCont_fwind[i]),cbind(seq(1,Ns),PG_fwind[,i]), by.x=1,by.y=1)[,2])
}
VaRCont_fwind
VaRfr1_fwind
VaRfr2_fwind
CVaRCont_fwind
CVaRfr1_fwind
CVaRfr2_fwind


#VaR y CVaR Total

#Sin alisado 
VaRTotal_fwind=equantile(PGT_fwind,1-alpha,Ns)
CVaRTotal_fwind= mean(merge(which(PGT_fwind<VaRTotal_fwind),cbind(seq(1,Ns),PGT_fwind), by.x=1,by.y=1)[,2])

VaRTotalfr1_fwind=equantile(PGfr1T_fwind,1-alpha,Ns)
CVaRTotalfr1_fwind= mean(PGfr1T_fwind[which(PGfr1T_fwind<VaRTotalfr1_fwind),])

VaRTotalfr2_fwind=equantile(PGfr2T_fwind,1-alpha,Ns)
CVaRTotalfr2_fwind= mean(PGfr2T_fwind[which(PGfr2T_fwind<VaRTotalfr2_fwind),])

VaRTotalfr3_fwind=equantile(PGfr3T_fwind,1-alpha,Ns)
CVaRTotalfr3_fwind= mean(PGfr3T_fwind[which(PGfr3T_fwind<VaRTotalfr3_fwind),])

#Con alisado 
VaRTotal_CA_fwind=wquantile(PGT_fwind,p_esc,1-alpha)
CVaRTotal_CA_fwind= mean(merge(which(PGT_fwind<VaRTotal_CA_fwind),cbind(seq(1,Ns),PGT_fwind), by.x=1,by.y=1)[,2])

VaRTotalfr1_CA_fwind=wquantile(PGfr1T_fwind,p_esc,1-alpha)
CVaRTotalfr1_CA_fwind= mean(PGfr1T_fwind[which(PGfr1T_fwind<VaRTotalfr1_CA_fwind),])

VaRTotalfr2_CA_fwind=wquantile(PGfr2T_fwind,p_esc,1-alpha)
CVaRTotalfr2_CA_fwind= mean(PGfr2T_fwind[which(PGfr2T_fwind<VaRTotalfr2_CA_fwind),])

VaRTotalfr3_CA_fwind=wquantile(PGfr3T_fwind,p_esc,1-alpha)
CVaRTotalfr3_CA_fwind= mean(PGfr3T_fwind[which(PGfr3T_fwind<VaRTotalfr3_CA_fwind),])


#Resumen
FWD_IND_VaR_CVaR <- rbind(c(VaRTotal_fwind,CVaRTotal_fwind, VaRTotal_CA_fwind, CVaRTotal_CA_fwind),
                          c(VaRTotalfr1_fwind,CVaRTotalfr1_fwind,VaRTotalfr1_CA_fwind,CVaRTotalfr1_CA_fwind),
                          c(VaRTotalfr2_fwind, CVaRTotalfr2_fwind, VaRTotalfr2_CA_fwind, CVaRTotalfr2_CA_fwind),
                          c(VaRTotalfr3_fwind,CVaRTotalfr3_fwind,VaRTotalfr3_CA_fwind,CVaRTotalfr3_CA_fwind)
                          )
colnames(FWD_IND_VaR_CVaR ) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(FWD_IND_VaR_CVaR ) <- c("Total Forward IND", "FR1", "FR2", "FR3")
FWD_IND_VaR_CVaR


                     VaR SA   CVaR SA    VaR CA   CVaR CA
Total Forward IND -45739.17 -58703.79 -14433.19 -27276.03
FR1               -45739.17 -58703.79 -14433.19 -27276.03
FR2               -45739.17 -58703.79 -14433.19 -27276.03
FR3               -45739.17 -58703.79 -14433.19 -27276.03


# Riesgo integral

In [298]:
%%R

#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
PGPort_ACC=PGfr2T_acc_div + PGfr3T_fwind #Pérdidas y ganancias

#Sin alisado
VaRPort_ACC=equantile(PGPort_ACC,1-alpha,Ns) #VaR
CVaRPort_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_ACC)]) #CVaR

#Con alisado
VaRPort_CA_ACC=wquantile(PGPort_ACC,p_esc, 1-alpha) #VaR
CVaRPort_CA_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_CA_ACC)]) #CVaR

#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc
PGPort_TI=PGT_bd+PGT_sw+PGfr1T_oir +PGfr2T_oir + PGfr1T_fwind +PGfr2T_fwind + PGfr1T_fwtdc + PGfr2T_fwtdc#Pérdidas y ganancias

#Sin alisado
VaRPort_TI=equantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR

#Con alisado
VaRPort_CA_TI=wquantile(PGPort_TI,p_esc, 1-alpha) #VaR
CVaRPort_CA_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_CA_TI)]) #CVaR

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc
PGPort_TDC=PGfr1T_acc_div + PGfr3T_fwtdc  #Pérdidas y ganancias

#Sin alisado
VaRPort_TDC=equantile(PGPort_TDC,1-alpha,Ns) #VaR
CVaRPort_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_TDC)]) #CVaR

#Con alisado
VaRPort_CA_TDC=wquantile(PGPort_TDC,p_esc,1-alpha) #VaR
CVaRPort_CA_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_CA_TDC)]) #CVaR

#Volatilidad
#1. Sólo aplica la volatilidad de Opciones de tasa de interés

PGPort_VOL=PGfr3T_oir  #Pérdidas y ganancias

#Sin alisado
VaRPort_VOL=equantile(PGPort_VOL,1-alpha,Ns) #VaR
CVaRPort_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_VOL)]) #CVaR

#Con alisado
VaRPort_CA_VOL=wquantile(PGPort_VOL,p_esc,1-alpha) #VaR
CVaRPort_CA_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_CA_VOL)]) #CVaR

#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos

PGT_Port=PGPort_ACC+PGPort_TI+PGPort_TDC+PGPort_VOL

VaRTotal_Port=equantile(PGT_Port,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]) #CVaR

VaRTotal_CA_Port=wquantile(PGT_Port,p_esc,1-alpha) #VaR
CVaRTotal_CA_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_CA_Port)]) #CVaR


#Resumen portafolio
FR_PORT_VaR_CVaR <- rbind(c(VaRPort_ACC,CVaRPort_ACC, VaRPort_CA_ACC, CVaRPort_CA_ACC),
                            c(VaRPort_TI, CVaRPort_TI, VaRPort_CA_TI, CVaRPort_CA_TI),
                            c(VaRPort_TDC, CVaRPort_TDC, VaRPort_CA_TDC, CVaRPort_CA_TDC),
                            c(VaRPort_VOL, CVaRPort_VOL,VaRPort_CA_VOL, CVaRPort_CA_VOL ),
                             c(VaRTotal_Port, CVaRTotal_Port,VaRTotal_CA_Port, CVaRTotal_CA_Port)
                          )
colnames(FR_PORT_VaR_CVaR ) <-  c("VaR SA", "CVaR SA", "VaR CA", "CVaR CA")
rownames(FR_PORT_VaR_CVaR ) <- c("Acciones", "Tasas de interés", "Divisas", "Volatilidad", "Portafolio")
FR_PORT_VaR_CVaR


                        VaR SA       CVaR SA        VaR CA       CVaR CA
Acciones         -4.702556e+04 -5.799282e+04 -1.513067e+04 -27682.296724
Tasas de interés -1.021558e+05 -1.151543e+05 -3.751433e+04 -67264.343365
Divisas          -2.722121e+02 -3.375619e+02 -4.081783e+02           NaN
Volatilidad      -1.818194e-02 -3.043336e-02 -7.568818e-03     -0.012913
Portafolio       -1.479291e+05 -1.730477e+05 -5.204207e+04 -92321.662149


# Conclusiones

Con un nivel de cofnianza del 98% hemos calculado el Valor en Riesgo y el Valor en Riesgo condicional para cada tipo de instrumento financiero, instrumento individual, factores de riesgo y para el portafolio, de los valores obtenidos podemos concluir lo siguiente.


Para la posición en **acciones** se espera una pérdida máxima de -47025.56, aproximadamente, que en general bastante grade pero hay que considerar que la posición que se tiene en el Futuro sobre el IPC es bastnte significativa.

La pérdida máxima esperada de las **divisas**, considerando el portafolio,  no es muy grande, y esto se debe a que se tienen posiciones y tipos de cambio variados y al considerar el total del portafolio, la pérdida se suaviza.

En el caso de los cetes, se observan pérdidas esperadas muy pequeñas, esto debido a que la volatilidad en las tasas gubernamentales no suelen ser muy significativas. Por otro lado en los bondes se tienen distintos rendimientos esperados dependiendo del factor del riesgo.

Para los **futuros** los valores varian bastante por la posición y el tipo de factor de riesgo de cada uno.

En los **swaps** se esperan pérdidas máximas según la posición y el factor de riesgo, pero en general podríamos decir que en el swap largo se esperan pérdidas mayores, a comparación del swap corto.

Para las **opciones europeas** se tienen tasas no tan variables pero las pérdidas se pueden maximizar por las posiciones.

Finalmente, el valor total del portafolio es de 561,162.67, de donde con una confianza del 98% se obtuvo un VaR de -147,929.10 , es decir 2 de cada 100 días se espera una pérdida de a lo más 147,929.10, que representa más del 25% del valor del portafolio.





